In [ ]:
from rfpipe import pipeline
# Updated for rfpipe version 1.3.1
from rfpipe import candidates
import rfpipe
import numpy as np 
import pylab as plt
import matplotlib
import logging
import glob
logger = logging.getLogger('rfpipe')
%matplotlib inline
from refine_utils import cd_refine

In [ ]:
params = {
        'axes.labelsize' : 14,
        'font.size' : 9,
        'legend.fontsize': 12,
        'xtick.labelsize': 12,
        'ytick.labelsize': 12,
        'text.usetex': False,
        'figure.figsize': [12, 10]
        }
matplotlib.rcParams.update(params)

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

# Set up data paths

In [ ]:
datadir = '/lustre/aoc/projects/fasttransients/staging/'

SDMmininame = 'realfast_18B-405_TEST_R3_A_cut.58665.73435530093_1562005608240'
miniSDMlist = glob.glob(datadir+SDMmininame)
workdir = '/lustre/aoc/observers/nm-10222/rfgpu/Refinement/nbs/'
gainpath = '/home/mchammer/evladata/telcal/'

In [ ]:
# datadir = '/lustre/aoc/projects/fasttransients/staging/'

# SDMmininame = 'realfast_19A-397.sb37006961.eb37092350.58715.036993946764_1566274015550'
# #'realfast_19A-397.sb37006961.eb37092350.58715.036993946764_1566270772899'
# miniSDMlist = glob.glob(datadir+SDMmininame)
# workdir = '/lustre/aoc/observers/nm-10222/rfgpu/Refinement/'
# gainpath = '/home/mchammer/evladata/telcal/'

In [ ]:
gainfile = []
for sdm in miniSDMlist:
    sdmname = sdm.split('/')[-1]
    datasetId = '{0}'.format('_'.join(sdmname.split('_')[1:-1]))
    # # set the paths to the gainfile
    gainname = datasetId + '.GN'
    logging.info('Searching for the gainfile {0} in {1}'.format(gainname, gainpath))
    for path, dirs, files in os.walk(gainpath):
        for f in filter(lambda x: gainname in x, files):
            gainfile.append(os.path.join(path, gainname))
            break

In [ ]:
# gainfile = ['/home/mchammer/evladata/telcal/2019/08/19A-397.sb37006961.eb37092350.58715.036993946764.GN']

In [ ]:
gainfile

# Inputs

In [ ]:
preffile = datadir+'realfast.yml' # Not being used currently
search_sigma = 7
refine = True
classify = True

# fine_dm_search = False

# if fine_dm_search:
#     dm = 1760
#     dm_steps = 100
#     dm_frac = 0.2

In [ ]:
prefs={'saveplots': True, 'savenoise': False, 'savesols': False,
       'savecandcollection': True, 'savecanddata': True,
       'applyonlineflags': True, 'fftmode': 'cuda', 'clustercands': (4,3), 'maxdm':500}

# if fine_dm_search:
#     logging.info('Setting DM array around DM {0}, from {1} to {2} in {3} steps'.
#                  format(dm, dm-dm_frac*dm, dm+dm_frac*dm, dm_steps))
#     prefs['dmarr'] = list(np.linspace(dm-dm_frac*dm, dm+dm_frac*dm, dm_steps))

# Searching all miniSDMs

In [ ]:
for index, sdm in enumerate(miniSDMlist):
    st = rfpipe.state.State(sdmfile=os.path.join(datadir, sdm), sdmscan=1, inprefs = prefs)#preffile = preffile, name='NRAOdefaultS')
    #inprefs = prefs)
    st.prefs.workdir = workdir
    st.prefs.gainfile = gainfile[index]
    st.prefs.sigma_image1 = search_sigma
    st.prefs.savecanddata = True
    
    cc = pipeline.pipeline_scan(st)

# Classify and generate refinement plots

In [ ]:
# Classify the generated pickles using FETCH and generate refinement plots
sdmname = miniSDMlist[0].split('/')[-1]
for pkl in glob.glob(st.prefs.workdir+'/'+'cands_*'+sdmname.split('/')[0]+'*.pkl'):
    if classify or refine:
        logging.info('Refining and classifying pkl: {0}'.format(pkl))
        ccs = list(candidates.iter_cands(pkl, select='candcollection'))
        for cc in ccs:
            cds = cc.canddata
            if cds:
                for cd in cds:
                    if classify:
                        payload = candidates.cd_to_fetch(cd, classify=True, save_png=True, show=True, mode = 'GPU', devicenum='0')
                        logging.info('FETCH FRB Probability of the candidate {0} is {1}'.format(cd.candid, payload))
                    if refine:
                        logging.info('Generating Refinement plots')
                        cd_refine(cd)
            else:
                logging.info('No candidate was found in cc: {0}'.format(cc))